In [1]:
!pip install open3d==0.17.0


[notice] A new release of pip is available: 23.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import numpy as np
import os

def create_pillow_mesh(width=2.0, height=0.5, depth=1.5, resolution=20):
    # Create a grid of vertices for the pillow surface
    x = np.linspace(-width/2, width/2, resolution)
    z = np.linspace(-depth/2, depth/2, resolution)
    x, z = np.meshgrid(x, z)
    
    # Create a slight bulge in the middle for the pillow effect
    y = height * (1 - ((x/(width/2))**2 + (z/(depth/2))**2)/2)
    
    # Create vertices
    vertices = []
    for i in range(resolution):
        for j in range(resolution):
            # Top surface
            vertices.append([x[i,j], y[i,j], z[i,j]])
            # Bottom surface
            vertices.append([x[i,j], -height/2, z[i,j]])
            
    vertices = np.array(vertices)
    
    # Create faces
    faces = []
    for i in range(resolution-1):
        for j in range(resolution-1):
            base = 2 * (i * resolution + j)
            # Top surface triangles
            faces.append([base, base+2, base+2*resolution])
            faces.append([base+2, base+2*resolution+2, base+2*resolution])
            # Bottom surface triangles
            faces.append([base+1, base+2*resolution+1, base+3])
            faces.append([base+3, base+2*resolution+1, base+2*resolution+3])
            
    faces = np.array(faces)
    return vertices, faces

def create_ball_mesh(radius=1.0, resolution=20):
    # Generate spherical coordinates
    phi = np.linspace(0, 2*np.pi, resolution)
    theta = np.linspace(0, np.pi, resolution)
    phi, theta = np.meshgrid(phi, theta)

    # Convert to Cartesian coordinates
    x = radius * np.sin(theta) * np.cos(phi)
    y = radius * np.sin(theta) * np.sin(phi)
    z = radius * np.cos(theta)

    # Create vertices
    vertices = np.stack([x.flatten(), y.flatten(), z.flatten()]).T
    
    # Create faces
    faces = []
    for i in range(resolution-1):
        for j in range(resolution-1):
            v0 = i * resolution + j
            v1 = v0 + 1
            v2 = (i + 1) * resolution + j
            v3 = v2 + 1
            faces.append([v0, v1, v2])
            faces.append([v1, v3, v2])
    
    return vertices, np.array(faces)

def save_obj(vertices, faces, filename):
    with open(filename, 'w') as f:
        # Write vertices
        for v in vertices:
            f.write(f'v {v[0]:.6f} {v[1]:.6f} {v[2]:.6f}\n')
        
        # Write faces (OBJ format uses 1-based indices)
        for face in faces:
            f.write(f'f {face[0]+1} {face[1]+1} {face[2]+1}\n')

# Create output directory
os.makedirs("outputs", exist_ok=True)

# Generate and save pillow mesh
pillow_vertices, pillow_faces = create_pillow_mesh()
save_obj(pillow_vertices, pillow_faces, "outputs/pillow.obj")

# Generate and save ball mesh
ball_vertices, ball_faces = create_ball_mesh()
save_obj(ball_vertices, ball_faces, "outputs/ball.obj")

# Print information about both meshes
print("Pillow Mesh Information:")
print(f"Number of vertices: {len(pillow_vertices)}")
print(f"Number of faces: {len(pillow_faces)}")
print("Model saved as: outputs/pillow.obj")

print("\nBall Mesh Information:")
print(f"Number of vertices: {len(ball_vertices)}")
print(f"Number of faces: {len(ball_faces)}")
print("Model saved as: outputs/ball.obj")

Pillow Mesh Information:
Number of vertices: 800
Number of faces: 1444
Model saved as: outputs/pillow.obj

Ball Mesh Information:
Number of vertices: 400
Number of faces: 722
Model saved as: outputs/ball.obj


In [3]:
# Let's verify the contents of both OBJ files
print("First few lines of pillow.obj:")
with open("outputs/pillow.obj", "r") as f:
    print("".join(f.readlines()[:10]))

print("\nFirst few lines of ball.obj:")
with open("outputs/ball.obj", "r") as f:
    print("".join(f.readlines()[:10]))

First few lines of pillow.obj:
v -1.000000 0.000000 -0.750000
v -1.000000 -0.250000 -0.750000
v -0.894737 0.049861 -0.750000
v -0.894737 -0.250000 -0.750000
v -0.789474 0.094183 -0.750000
v -0.789474 -0.250000 -0.750000
v -0.684211 0.132964 -0.750000
v -0.684211 -0.250000 -0.750000
v -0.578947 0.166205 -0.750000
v -0.578947 -0.250000 -0.750000


First few lines of ball.obj:
v 0.000000 0.000000 1.000000
v 0.000000 0.000000 1.000000
v 0.000000 0.000000 1.000000
v 0.000000 0.000000 1.000000
v 0.000000 0.000000 1.000000
v -0.000000 0.000000 1.000000
v -0.000000 0.000000 1.000000
v -0.000000 0.000000 1.000000
v -0.000000 0.000000 1.000000
v -0.000000 0.000000 1.000000



In [4]:
readme_content = """# 3D Mesh Generator

This Python script generates 3D meshes for two simple objects: a pillow and a ball. The meshes are saved in Wavefront OBJ format.

## Features

- Creates a pillow mesh with a curved top surface and flat bottom
- Creates a spherical ball mesh
- Exports meshes to standard OBJ format
- Configurable parameters for size and resolution

## Generated Meshes

1. Pillow (outputs/pillow.obj):
   - Dimensions: 2.0 x 0.5 x 1.5 (width x height x depth)
   - Vertices: 800
   - Faces: 1,444
   - Features a subtle bulge in the middle for realistic pillow effect

2. Ball (outputs/ball.obj):
   - Radius: 1.0
   - Vertices: 400
   - Faces: 722
   - Perfect sphere generated using spherical coordinates

## Functions

- `create_pillow_mesh(width=2.0, height=0.5, depth=1.5, resolution=20)`
- `create_ball_mesh(radius=1.0, resolution=20)`
- `save_obj(vertices, faces, filename)`

## Output Format

The meshes are saved in standard OBJ format:
- Vertices are defined with 'v' followed by x, y, z coordinates
- Faces are defined with 'f' followed by vertex indices (1-based)

## Dependencies

- NumPy
- os (standard library)
"""

# Save the README
with open("README.md", "w") as f:
    f.write(readme_content)

print("README.md has been created. Contents:")
print("\n" + readme_content)

README.md has been created. Contents:

# 3D Mesh Generator

This Python script generates 3D meshes for two simple objects: a pillow and a ball. The meshes are saved in Wavefront OBJ format.

## Features

- Creates a pillow mesh with a curved top surface and flat bottom
- Creates a spherical ball mesh
- Exports meshes to standard OBJ format
- Configurable parameters for size and resolution

## Generated Meshes

1. Pillow (outputs/pillow.obj):
   - Dimensions: 2.0 x 0.5 x 1.5 (width x height x depth)
   - Vertices: 800
   - Faces: 1,444
   - Features a subtle bulge in the middle for realistic pillow effect

2. Ball (outputs/ball.obj):
   - Radius: 1.0
   - Vertices: 400
   - Faces: 722
   - Perfect sphere generated using spherical coordinates

## Functions

- `create_pillow_mesh(width=2.0, height=0.5, depth=1.5, resolution=20)`
- `create_ball_mesh(radius=1.0, resolution=20)`
- `save_obj(vertices, faces, filename)`

## Output Format

The meshes are saved in standard OBJ format:
- Vertices

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=00b865fd-77a3-4483-834c-d0a9acf21d20' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>